In [1]:
# !pip install langchain_chroma
# !pip install langchain_huggingface
# !pip install langchain_openai

In [2]:
# !pip install protobuf==3.20.*

In [3]:
from agent import Chatbot, Text2SQL

In [4]:
from agent.const import (
    ChatConfig,
    Text2SQLConfig,
    GEMINI_FAST_CONFIG,
    TEXT2SQL_MEDIUM_OPENAI_CONFIG,
    TEXT2SQL_MEDIUM_GEMINI_CONFIG,
    TEXT2SQL_FASTEST_CONFIG,
    TEXT2SQL_SWEET_SPOT_CONFIG
)

from agent.prompt.prompt_controller import (
    PromptConfig, 
    VERTICAL_PROMPT_BASE, 
    VERTICAL_PROMPT_UNIVERSAL,
    HORIZONTAL_PROMPT_BASE,
    HORIZONTAL_PROMPT_UNIVERSAL
)

In [5]:
from ETL.dbmanager.setup import (
    DBConfig,
    BGE_VERTICAL_BASE_CONFIG,
    BGE_VERTICAL_UNIVERSAL_CONFIG,
    BGE_HORIZONTAL_BASE_CONFIG,
    setup_db
)

from langchain_huggingface import HuggingFaceEmbeddings
import json
import torch

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [6]:
db_config = DBConfig(**BGE_VERTICAL_UNIVERSAL_CONFIG)
chat_config = ChatConfig(**GEMINI_FAST_CONFIG)
text2sql_config = Text2SQLConfig(**TEXT2SQL_MEDIUM_OPENAI_CONFIG)
prompt_config = PromptConfig(**VERTICAL_PROMPT_UNIVERSAL)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5', model_kwargs = {'device': device})
db_config.embedding = embedding_model

db = setup_db(db_config)
text2sql = Text2SQL(config = text2sql_config, prompt_config=prompt_config, db = db, max_steps=2)

2024-12-18 13:33:41,979 - INFO - PyTorch version 2.2.0+cu121 available.
2024-12-18 13:33:41,982 - INFO - TensorFlow version 2.10.1 available.
2024-12-18 13:33:42,538 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-12-18 13:33:47,071 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


test_db


2024-12-18 13:33:47,261 - INFO - Finish setup for Vertical Universal


In [8]:
chatbot = Chatbot(config = chat_config, text2sql = text2sql)